In [1]:
%matplotlib ipympl
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import critical_loads as cl
import statsmodels.formula.api as sm

plt.style.use('ggplot')

# Critcal Loads: Dockerised workflow (November 2018)

This notebook begins the process of transferring the Critical Loads workflow out of RESA and into Docker, so that it can eventually be moved onto the new data platform. A workflow is provided for creating a new PostGIS database in Docker and populating it with key datasets required for the critical loads calculations. Some data exploration and checking is also presented.

## 1. Create a PostGIS database running on Docker

The following code has been used to create a PostGIS database running on my (local) Docker host.

    # Create data volume
    docker volume create pg_data_crit_lds
    
    # Create container
    docker run --name=postgis_crit_lds -d -e POSTGRES_USER=admin -e POSTGRES_PASS=pw1234 -e POSTGRES_DBNAME=critical_loads -e ALLOW_IP_RANGE=0.0.0.0/0 -p 25432:5432 -v pg_data_crit_lds:/var/lib/postgresql kartoza/postgis:9.6-2.4
    
**Note:** Once the container has been created, it can be stopped/started at any time using

    # Start
    docker start postgis_crit_lds
    
    # Stop
    docker stop postgis_crit_lds
    
We can now start the DS Toolkit and establish connections to the following databases:

 1. **RESA2**. A centralised NIVA database containing most of the old (non-spatial) data relating to the Critical Loads project
 
 2. **PostGIS running on `localhost`**. My local PostGIS installation, which stores some of the new spatial datasets required for the updated calculations
 
 3. **PostGIS running on Docker**. The new database created for the updated workflow. We wish to transfer all the Critical Loads datasets (both spatial and non-spatial) to this database

In [2]:
# Connect to the NIVABASE
ora_eng = nivapy.da.connect()

Connection successful.


In [3]:
# Connect to local PostGIS
loc_pg_eng = nivapy.da.connect(src='postgres',
                               db_name='niva_work',
                               port=5432)

Connection successful.


In [4]:
# Connect to Docker PostGIS
doc_pg_eng = nivapy.da.connect(src='postgres',
                               db_name='critical_loads',
                               port = 25432)

Connection successful.


## 2. Deposition data

First, create a new schema to store deposition data.

In [5]:
# Create deposition schema
sql = "CREATE SCHEMA deposition"
doc_pg_eng.execute(sql)

### 2.1. Transfer BLR and 0.1 degree deposition grids

Deposition data from NILU is now supplied on a 0.1 degree grid, which is currently stored in my local PostGIS installation. The code below transfers it to the new schema in Docker.

In [6]:
# Get 0.1 degree dep grid from PostGIS
sql = ("SELECT * FROM public.dep_grid_0_1deg")
dep_gdf = gpd.read_postgis(sql, loc_pg_eng)

# Write to new db
nivapy.da.gdf_to_postgis(dep_gdf, 
                         'dep_grid_0_1deg', 
                         'deposition', 
                         doc_pg_eng,
                         'dep_dep_grid_0_1deg_spidx',
                         if_exists='replace',
                         index=False)

# Use 'cell_id' col as primary key
sql = ("ALTER TABLE deposition.dep_grid_0_1deg "
       "ADD CONSTRAINT dep_grid_0_1deg_pk "
       "PRIMARY KEY (cell_id)")
doc_pg_eng.execute(sql)

In previous years, data have been supplied using the BLR grid, which is available as a shapefile. The code below loads this into the database as well.

In [7]:
# Read BLR .shp
blr_path = (r'../../../../GIS/shapefiles/blrgrid_uten_grums.shp')
dep_gdf = gpd.read_file(blr_path)
dep_gdf.rename({'BLR':'blr'},
               axis=1,
               inplace=True)

# Write to new db
nivapy.da.gdf_to_postgis(dep_gdf, 
                         'dep_grid_blr', 
                         'deposition', 
                         doc_pg_eng,
                         'dep_dep_grid_blr_spidx',
                         if_exists='replace',
                         index=False)

# Use 'blr' col as primary key
sql = ("ALTER TABLE deposition.dep_grid_blr "
       "ADD CONSTRAINT dep_grid_blr_pk "
       "PRIMARY KEY (blr)")
doc_pg_eng.execute(sql)

WARNING! The dataframe contains mixed geometries:
  {'MultiPolygon', 'Polygon'}
These will be cast to "Multi" type. If this is not what you want, consider using gdf.explode() first


### 2.2. Transfer "Deposition Series Definitions" from the NIVABASE

In RESA2, the table `DEP_SERIES_DEFINITIONS` identifies the datasets previously supplied by NILU. Each dataset has a separate row (and ID) in this table. For consistency, I will transfer this table directly and use the same dataset IDs in the new workflow.

In [8]:
# Read data from RESA
sql = "SELECT * FROM resa2.dep_series_definitions"
df = pd.read_sql(sql, ora_eng)

# Tidy
df.rename({'dep_series_id':'series_id'},
          axis=1,
          inplace=True)
df.sort_values('series_id', inplace=True)
df.tail()

,series_id,name,description
14,24,expost gridav MFR2020-Update,Lagt inn i forb. med ex-Post analyser 2010 For...
15,25,Middel 2007-2011,Fordelt til BLR av NILU 2012 (Wenche Aas)
27,26,Middel 2012-2016,Fordelt til BLR av NILU 2017 (Wenche Aas; old ...
16,27,Middel 2012-2016 (new),Fordelt til BLR av NILU 2017 (Wenche Aas; new ...
17,28,Middel 2012-2016 (new; hi-res),Fordelt til BLR av NILU 2017 (Wenche Aas; new ...


In [9]:
# Write to new db
df.to_sql('dep_series_defs', 
          doc_pg_eng,
          'deposition',
          if_exists='replace',
          index=False)

# Use 'dep_series_id' col as primary key
sql = ("ALTER TABLE deposition.dep_series_defs "
       "ADD CONSTRAINT dep_series_defs_pk "
       "PRIMARY KEY (series_id)")
doc_pg_eng.execute(sql)

### 2.3. Transfer "Deposition Parameter Definitions" from the NIVABASE

In RESA2, the table `AIR_PARAMETER_DEFINITIONS` identifies various deposition parameters. As far as I can see, only the first three or four entries in this table are currently relevant. Many of the columns also seem unnecessary at present

In [10]:
# Read data from RESA
sql = ("SELECT * FROM resa2.air_parameter_definitions "
       "WHERE parameter_id < 5")
df = pd.read_sql(sql, ora_eng)

# Tidy
df.sort_values('parameter_id', inplace=True)
df.drop(['formula', 'category', 'function', 'entered_by', 'entered_date'],
        axis=1,
        inplace=True)
df.rename({'parameter_id':'param_id'}, 
          inplace=True,
          axis=1)
df

,param_id,name,unit,description
0,1,N(oks),mg N/m2/year,Oxidized nitrogen
1,2,N(red),mg N/m2/year,Reduced nitogen
2,3,S,mg S/m2/year,Total sulphur
3,4,S*,mg S/m2/year,Non marine sulphur


In [11]:
# Write to new db
df.to_sql('dep_param_defs', 
          doc_pg_eng,
          'deposition',
          if_exists='replace',
          index=False)

# Use 'dep_series_id' col as primary key
sql = ("ALTER TABLE deposition.dep_param_defs "
       "ADD CONSTRAINT dep_param_defs_pk "
       "PRIMARY KEY (param_id)")
doc_pg_eng.execute(sql)

### 2.4. Data for "Deposition Values"

#### 2.4.1. Create tables

The old workflow used the coarse BLR grid for deposition values. Deposition `series_ids` from 1 to 27 are all based on this grid, so these values will be transferred to a new table named `'dep_values_blr_grid'`. Deposition `series_id=28` uses the new 0.1 degree grid, so this dataset (and subsequent ones) will need adding to a separate table (`'dep_values_0_1deg_grid'`). As future data will also be delivered using this grid, it will be useful to have a function for processing and adding raw data to this table.

Additionally, both these tables need constraints to ensure that relevant series and parameter IDs are defined before uploading data. Note that I have not included a constraint on the BLR/cell ID, because the data supplied by NILU often includes cells that are outside of terrestrial Norway.

In [12]:
# Delete if already exist
sql = ("DROP TABLE IF EXISTS deposition.dep_values_blr_grid, "
       "  deposition.dep_values_0_1deg_grid")
doc_pg_eng.execute(sql)

# Create table for BLR data
sql = ("CREATE TABLE deposition.dep_values_blr_grid "
       "( "
       "  series_id integer NOT NULL, "
       "  blr integer NOT NULL, "
       "  param_id integer NOT NULL, "
       "  value numeric, "
       "  PRIMARY KEY (series_id, blr, param_id), "
       "  CONSTRAINT series_id_fkey FOREIGN KEY (series_id) "
       "      REFERENCES deposition.dep_series_defs (series_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION, "
       "  CONSTRAINT param_id_fkey FOREIGN KEY (param_id) "
       "      REFERENCES deposition.dep_param_defs (param_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION "
       ")")
doc_pg_eng.execute(sql)

# Create table for 0.1 degree data
sql = ("CREATE TABLE deposition.dep_values_0_1deg_grid "
       "( "
       "  series_id integer NOT NULL, "
       "  cell_id integer NOT NULL, "
       "  param_id integer NOT NULL, "
       "  value numeric, "
       "  PRIMARY KEY (series_id, cell_id, param_id), "
       "  CONSTRAINT series_id_fkey FOREIGN KEY (series_id) "
       "      REFERENCES deposition.dep_series_defs (series_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION, "
       "  CONSTRAINT param_id_fkey FOREIGN KEY (param_id) "
       "      REFERENCES deposition.dep_param_defs (param_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION "
       ")")
doc_pg_eng.execute(sql)

#### 2.4.2. Transfer "old" data

The old data using the BLR grid are stored in RESA2 in the table `DEP_BLR_VALUES`. We'll just transfer the data for `param_ids` 1 to 4.

In [13]:
# Read data from RESA
sql = ("SELECT * FROM resa2.dep_blr_values "
       "WHERE parameter_id < 5")
df = pd.read_sql(sql, ora_eng)

# Tidy
df.rename({'dep_series_id':'series_id',
           'parameter_id':'param_id'},
          axis=1,
          inplace=True)

# Write to new db
df.to_sql('dep_values_blr_grid', 
          doc_pg_eng,
          'deposition',
          if_exists='append',
          index=False)

#### 2.4.3. Import "new" data

In 2017, NILU supplied raw data for the 0.1 degree grid in `.dat` format. I previously wrote some code ([here](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow_new_grid.ipynb#1.1.-Upload-new-data-to-database)) to process this data. I have now generalised this and moved it into a new `.py` file, which will eventually become a Python package/module for the Critical Loads workflow.

**Note:** Double-check with Kari that the NILU file `tot_s_combined-emep_grid.dat` contains data for *non-marine* S. We assumed this in the 2018 analysis, but worth checking.

In [14]:
# Process NILU data and add to db
nilu_fold = r'../../../../raw_data/2012_2016'
df = cl.upload_nilu_0_1deg_dep_data(nilu_fold, 
                                    doc_pg_eng,
                                    28)
df.head()

,cell_id,param_id,value,series_id
0,50050305,2,270.87,28
1,50050315,2,240.50,28
2,50050325,2,259.19,28
3,50050335,2,252.06,28
4,50050345,2,332.82,28


## 3. Exceedance for vegatation

First, create a new schema for data relating to the vegetation calculations.

In [15]:
# Create veg schema
sql = "CREATE SCHEMA vegetation"
doc_pg_eng.execute(sql)

### 3.1. Transfer critical loads table for vegetation

The Excel file 

    Critical_Loads\vegetation\sat_veg_land_use_classes.xlsx
    
contains critical load values for various vegetation classes. The original is in the sheet named `'EUNIS_tilGIS'`, but I have also created a tidied version (with e.g. lower case column names) in the sheet named `'eunis_lower_case'`.

As above, if this table is likely to change, it should be normalised and loaded into the database more carefully. For now, the "flat" Excel format is convenient, but I nevertheless want to store it in the database with the rest of the data.

In [16]:
# Read Excel data
xl_path = r'../../../../vegetation/sat_veg_land_use_classes.xlsx'
df = pd.read_excel(xl_path, sheet_name='eunis_lower_case')

# Write to new db
df.to_sql('land_class_crit_lds', 
          doc_pg_eng,
          'vegetation',
          if_exists='replace',
          index=False)

# Use 'norut_code' as primary key
sql = ("ALTER TABLE vegetation.land_class_crit_lds "
       "ADD CONSTRAINT veg_land_class_crit_lds_pk "
       "PRIMARY KEY (norut_code)")
doc_pg_eng.execute(sql)

### 3.2. Transfer "critical loads for vegetation" grid

The code in Section 1.3 of [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow_new_grid.ipynb#1.3.-Reclassify) converted 30 m resolution satellite vegetation data into a grid of critical loads (based on the Excel table above - Section 3.1). Unless we update the vegetation data, this calculation does not need to be performed again, but it would be useful to store the resulting raster grid (`sat_veg_30m_cr_lds_div100.tif`) in the PostGIS database.  

It is not yet possible to load raster data via the Toolkit, because I haven't installed PostGIS and all the associated extensions in the container. However, I do have PostGIS installed on the Docker host, so I can load data into Docker PostGIS from there. The procedure is as follows:

 1. Open a Windows `CMD` prompt and `cd` into
 
         C:\Program Files\PostgreSQL\9.6\bin 
         
    <br> (or add `raster2pgsql` to your `PATH`).
    
 2. Run the following code
 
         raster2pgsql -s 32633 -d -C -I -M -l 2,4,8,16,32,64,128,256 full\path\to\raster.tif -t 100x100 vegetation.sat_veg_30m_all | psql -h localhost -U admin -p 25432 -d critical_loads
    
    <br>You will be prompted to enter the password for the Docker database. This command specifies the following options (more details [here](https://postgis.net/docs/using_raster_dataman.html)):
    
     * **-s**. EPSG code for spatial reference
     * **-d**. Drop ratser if already exists in database
     * **-C**. Apply raster constraints -- srid, pixelsize etc. to ensure raster is properly registered
     * **-I**. Build a spatial index
     * **-M**. Vaccum analyze after loading
     * **-l**. Build pyramids (8 levels)
     * **-t**. Cut raster into tiles to be inserted one per table row
 
    <br>The part after `psql` then specifies the connection details for the Docker database.     
   
**Note:** PostGIS can store very large rasters, but exploring them with QGIS via DB Manager can cause problems - either the database connection gets repeatedly interrupted, or QGIS crashes. NivaPy includes functions for reading PostGIS raster data into either (i) Numpy arrays, using `nivapy.da.postgis_raster_to_array()`, or (ii) GeoTiffs, using `nivapy.da.postgis_raster_to_geotiff()` (the latter function returns the data as a Numpy array as well). For large datasets, it may be better to export the data as a GeoTiff and then add it to QGIS. Alternatively, perform any spatial operations *within the database* first to generate a small set of results, then visualise these via DB Manager in QGIS.

## XXX. Exceedance for water

First, create a new schema for data relating to the water calculations.

In [18]:
# Create water schema
sql = "CREATE SCHEMA water"
doc_pg_eng.execute(sql)

### XXX.1. MAGIC model output for regression

Kari has a spreadsheet here

    K:\Avdeling\317 Klima- og miljømodellering\KAU\Focal Centre\Data\bc0regresjonNOK_TL2005-rapport_KAU.xls
    
containing output from the MAGIC model. This is used to generate parameters for the calculation of critical loads to water. It is unclear whether this data will be updated in the future. If so, the dataset should be properly normalised before adding to the database. However, for now I'm assuming that this is a static dataset, so I'm just athe essential data to the database as a "flat" table for the purposes of data storage.

In [19]:
# Read tidied Excel data
xl_path = r'../../../../water/update_nov_2018/regression_data_tidied.xls'
df = pd.read_excel(xl_path, sheet_name='tidied')

# Write to new db
df.to_sql('magic_regression_data', 
          doc_pg_eng,
          'water',
          if_exists='replace',
          index=False)

# Use ('resa_stn_id', 'sim_yr') as primary key
sql = ("ALTER TABLE water.magic_regression_data "
       "ADD CONSTRAINT water_magic_regression_data_pk "
       "PRIMARY KEY (resa_stn_id, sim_yr)")
doc_pg_eng.execute(sql)

### XXX.2. Perform regression

In the MAGIC data table, values for `bc_x_k` in 1860 are used to define $BC^*_0$, and values from 1986 are used to define $BC^*$. The regression uses $BC^*$ as the x-variable and $BC^*_0$ as the y-variable.

In [20]:
# Get data
sql = ("SELECT resa_stn_id, sim_yr, bc_x_k "
       "FROM water.magic_regression_data")
df = pd.read_sql(sql, doc_pg_eng)
df.index = df['resa_stn_id']
del df['resa_stn_id']

# Split by year
bc0_df = df.query('sim_yr == 1860')
del bc0_df['sim_yr']
bc0_df.columns = ['BC0']

bc_df = df.query('sim_yr == 1986')
del bc_df['sim_yr']
bc_df.columns = ['BC']

# Join
df = bc0_df.join(bc_df)

# Regression
res = sm.ols(formula='BC0 ~ BC', data=df).fit()

print (res.summary())

                            OLS Regression Results                            
Dep. Variable:                    BC0   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     5320.
Date:                Wed, 21 Nov 2018   Prob (F-statistic):           1.20e-75
Time:                        14:05:57   Log-Likelihood:                -211.76
No. Observations:                  83   AIC:                             427.5
Df Residuals:                      81   BIC:                             432.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2744      0.536      0.512      0.6

This is the same result as in Kari's Excel spreadsheet. Note, however, that the intercept is not significantly different from zero. This implies an alternative, "slope-only" model *might* be better.

In [21]:
# Regression
res = sm.ols(formula='BC0 ~ BC - 1', data=df).fit()
print (res.summary())

                            OLS Regression Results                            
Dep. Variable:                    BC0   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.310e+04
Date:                Wed, 21 Nov 2018   Prob (F-statistic):           3.12e-92
Time:                        14:06:00   Log-Likelihood:                -211.89
No. Observations:                  83   AIC:                             425.8
Df Residuals:                      82   BIC:                             428.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
BC             0.9481      0.008    114.449      0.0

This model is better (lower AIC and BIC) than the original, and it's also simpler. I therefore propose using

$$BC^*_0 = 0.9481BC^*_t$$

instead of 

$$BC^*_0 = 0.9431BC^*_t + 0.2744$$